# TLS 1.3 handshake overview
This is the basic TLS 1.3 handshake:

<img src="../notebooks/images/handshake_tls13.png" alt="Handshake TLS 1.3" width="400"/>

In [1]:
from scapy.all import *

In [2]:
record1_str = open('../notebooks/raw_data/tls_session_13/01_cli.raw').read()
record1 = TLS(record1_str)
sess = record1.tls_session
record1.show()

###[ TLS ]### 
  type      = handshake
  version   = TLS 1.0
  len       = 174
  iv        = ''
  \msg       \
   |###[ TLS Handshake - Client Hello ]### 
   |  msgtype   = client_hello
   |  msglen    = 170
   |  version   = TLS 1.2
   |  gmt_unix_time= Wed, 07 Nov 2085 08:42:53 +0000 (3655960973)
   |  random_bytes= f63d43adbe64a2634f0b63bcdc4019a3e526bc013a6042e05b14555c
   |  sidlen    = 0
   |  sid       = ''
   |  cipherslen= 6
   |  ciphers   = [TLS_AES_128_GCM_SHA256, TLS_CHACHA20_POLY1305_SHA256, TLS_AES_256_GCM_SHA384]
   |  complen   = 1
   |  comp      = null
   |  extlen    = 123
   |  \ext       \
   |   |###[ TLS Extension - Server Name ]### 
   |   |  type      = server_name
   |   |  len       = 11
   |   |  servernameslen= 9
   |   |  servernames= [server]
   |   |###[ TLS Extension - Renegotiation Indication ]### 
   |   |  type      = renegotiation_info
   |   |  len       = 1
   |   |  reneg_conn_len= 0
   |   |  renegotiated_connection= ''
   |   |###[ TLS Extensi

In [3]:
record2_str = open('../notebooks/raw_data/tls_session_13/02_srv.raw').read()
record2 = TLS(record2_str, tls_session=sess.mirror())
record2.show()

###[ TLS ]### 
  type      = handshake
  version   = TLS 1.0
  len       = 14
  iv        = ''
  \msg       \
   |###[ TLS 1.3 Handshake - Hello Retry Request ]### 
   |  msgtype   = hello_retry_request
   |  msglen    = 10
   |  version   = TLS 1.3-d18
   |  extlen    = 6
   |  \ext       \
   |   |###[ TLS Extension - Key Share (for HelloRetryRequest) ]### 
   |   |  type      = key_share
   |   |  len       = 2
   |   |  selected_group= secp256r1
  mac       = ''
  pad       = ''
  padlen    = None



In [4]:
record3_str = open('../notebooks/raw_data/tls_session_13/03_cli.raw').read()
record3 = TLS(record3_str, tls_session=sess.mirror())
record3.show()

###[ TLS ]### 
  type      = handshake
  version   = TLS 1.0
  len       = 207
  iv        = ''
  \msg       \
   |###[ TLS Handshake - Client Hello ]### 
   |  msgtype   = client_hello
   |  msglen    = 203
   |  version   = TLS 1.2
   |  gmt_unix_time= Wed, 07 Nov 2085 08:42:53 +0000 (3655960973)
   |  random_bytes= f63d43adbe64a2634f0b63bcdc4019a3e526bc013a6042e05b14555c
   |  sidlen    = 0
   |  sid       = ''
   |  cipherslen= 6
   |  ciphers   = [TLS_AES_128_GCM_SHA256, TLS_CHACHA20_POLY1305_SHA256, TLS_AES_256_GCM_SHA384]
   |  complen   = 1
   |  comp      = null
   |  extlen    = 156
   |  \ext       \
   |   |###[ TLS Extension - Server Name ]### 
   |   |  type      = server_name
   |   |  len       = 11
   |   |  servernameslen= 9
   |   |  servernames= [server]
   |   |###[ TLS Extension - Renegotiation Indication ]### 
   |   |  type      = renegotiation_info
   |   |  len       = 1
   |   |  reneg_conn_len= 0
   |   |  renegotiated_connection= ''
   |   |###[ TLS Extensi

In [5]:
# The PFS relies on the ECDH secret below being kept from observers, and deleted right after the key exchange
from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePrivateNumbers
from cryptography.hazmat.backends import default_backend
secp256r1_client_privkey = open('../notebooks/raw_data/tls_session_13/cli_key.raw').read()
pubnum = sess.tls13_client_pubshares["secp256r1"].public_numbers()
privnum = EllipticCurvePrivateNumbers(pkcs_os2ip(secp256r1_client_privkey), pubnum)
privkey = privnum.private_key(default_backend())
sess.tls13_client_privshares["secp256r1"] = privkey

In [6]:
record4_str = open('../notebooks/raw_data/tls_session_13/04_srv.raw').read()
record4 = TLS(record4_str, tls_session=sess.mirror())
record4.show()

###[ TLS ]### 
  type      = handshake
  version   = TLS 1.0
  len       = 115
  iv        = ''
  \msg       \
   |###[ TLS 1.3 Handshake - Server Hello ]### 
   |  msgtype   = server_hello
   |  msglen    = 111
   |  version   = TLS 1.3-d18
   |  random_bytes= 96ff693075d8465651a9c28773f5496542206ba390199b9c997545d9a1266615
   |  cipher    = TLS_AES_128_GCM_SHA256
   |  extlen    = 73
   |  \ext       \
   |   |###[ TLS Extension - Key Share (for ServerHello) ]### 
   |   |  type      = key_share
   |   |  len       = 69
   |   |  \server_share\
   |   |   |###[ Key Share Entry ]### 
   |   |   |  group     = secp256r1
   |   |   |  kxlen     = 65
   |   |   |  key_exchange= '\x04\x8aM\t\xcd\xe5\x8d\xbc\x04\x19U\xb9\xa4\x1aC\xc1im\xc5B\x9f\xfa\x96\xf9\xcd\x19J\x86:\xc7\x82\xf1\x81Y\xf0r\xb4\xf6\x10!]\x86@}\xd76\x8buJ\xb2\xe6O,\x1b?\x9dE|&N+\x17\x81\xa3k'
  mac       = ''
  pad       = ''
  padlen    = None



In [7]:
record5_str = open('../notebooks/raw_data/tls_session_13/05_srv.raw').read()
record5 = TLS(record5_str, tls_session=sess)
record5.show()

###[ TLS 1.3 ]### 
  type      = application_data
  version   = TLS 1.0
  len       = 656    [deciphered_len= 640]
  \inner     \
   |###[ TLS Inner Plaintext ]### 
   |  \msg       \
   |   |###[ TLS 1.3 Handshake - Encrypted Extensions ]### 
   |   |  msgtype   = encrypted_extensions
   |   |  msglen    = 18
   |   |  extlen    = 16
   |   |  \ext       \
   |   |   |###[ TLS Extension - Supported Groups ]### 
   |   |   |  type      = supported_groups
   |   |   |  len       = 8
   |   |   |  groupslen = 6
   |   |   |  groups    = [secp256r1, secp384r1, x25519]
   |   |   |###[ TLS Extension - Server Name ]### 
   |   |   |  type      = server_name
   |   |   |  len       = 0
   |   |   |  servernameslen= None
   |   |   |  servernames= []
   |   |###[ TLS 1.3 Handshake - Certificate ]### 
   |   |  msgtype   = certificate
   |   |  msglen    = 441
   |   |  cert_req_ctxt_len= 0
   |   |  cert_req_ctxt= ''
   |   |  certslen  = 437
   |   |  \certs     \
   |   |   |###[ Certificat

In [8]:
record6_str = open('../notebooks/raw_data/tls_session_13/06_cli.raw').read()
record6 = TLS(record6_str, tls_session=sess.mirror())
record6.show()

###[ TLS 1.3 ]### 
  type      = application_data
  version   = TLS 1.0
  len       = 53    [deciphered_len= 37]
  \inner     \
   |###[ TLS Inner Plaintext ]### 
   |  \msg       \
   |   |###[ TLS Handshake - Finished ]### 
   |   |  msgtype   = finished
   |   |  msglen    = 32
   |   |  vdata     = "\xfcS\xf1\xbe\xb1Co\xb9h1\x92\x99\xa9\xe3\xb6\x08\x8c\xc9\x9eB\x17\x8cw3{\xc5'\x86\xd0\x84\x88/"
   |  type      = handshake
   |  pad       = ''
  auth_tag  = '\xe8\xb04-e\xdb\x8a\x7f\x07N\xedR~\xa6\x98\xa6'



## Observations sur TLS 1.3
* Certificat désormais chiffré...
* ...mais pas le Server Name dans le ClientHello
* Risques du mode 0-RTT